In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
DF = "Weatherdata.csv"
Data = pd.read_csv(DF)
Data.dropna(inplace = True)
Data.head()

,City,Latitude,Longitude,Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date and Time
0,Sobolevo,54.4299,31.8996,42.96,70,12,5.73,RU,2022-05-22 19:21:36
1,Ribeira Grande,38.5167,-28.7000,64.80,100,75,2.30,PT,2022-05-22 19:21:37
2,Amahai,-3.3333,128.9167,77.83,87,99,1.90,ID,2022-05-22 19:21:37
3,Nanortalik,60.1432,-45.2371,37.51,98,100,6.85,GL,2022-05-22 19:21:37
4,Kenitra,34.2610,-6.5802,68.76,77,48,7.49,MA,2022-05-22 19:20:17


In [3]:
gmaps.configure(g_key)
#Gather coordinates and humidity
Coordinates = Data[["Latitude","Longitude"]]
Humidity = Data["Humidity"].astype(float)


In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

# Create heat layer
Hum_layer = gmaps.heatmap_layer(Coordinates, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(Hum_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
Conditions = Data.loc[(Data["Maximum Temperature (F)"] < 65) & 
                      (Data["Maximum Temperature (F)"] > 35) & (Data["Cloudiness"] > 20) & (Data["Wind Speed"] < 10)
                     & (Data["Humidity"] < 60)]
Conditions = Conditions.dropna()
Conditions.head(10)

,City,Latitude,Longitude,Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date and Time
71,Norman Wells,65.2820,-126.8329,44.64,49,75,5.75,CA,2022-05-22 19:21:59
79,Piranshahr,36.7010,45.1413,60.35,42,59,3.87,IR,2022-05-22 19:22:02
166,Griffith,41.5284,-87.4237,63.05,48,100,4.61,US,2022-05-22 19:22:34
168,Fairbanks,64.8378,-147.7164,64.36,30,40,3.44,US,2022-05-22 19:21:22
184,Kosh-Agach,50.0000,88.6667,57.52,41,100,7.29,RU,2022-05-22 19:22:40
196,Rocha,-34.4833,-54.3333,64.58,58,22,2.86,UY,2022-05-22 19:22:44
199,Haines Junction,60.7522,-137.5108,57.15,30,100,3.44,CA,2022-05-22 19:22:45
227,Srednekolymsk,67.4500,153.6833,50.56,51,99,9.69,RU,2022-05-22 19:22:54
303,College,64.8569,-147.8028,64.35,29,40,9.22,US,2022-05-22 19:23:23
312,Kalabo,-14.9700,22.6814,57.61,30,77,6.04,ZM,2022-05-22 19:23:27


In [6]:
Hotel_df = Data

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in Hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        Hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No hotel noted.")

Retrieving Results for Index 0: Sobolevo.
No hotel noted.
Retrieving Results for Index 1: Ribeira Grande.
No hotel noted.
Retrieving Results for Index 2: Amahai.
No hotel noted.
Retrieving Results for Index 3: Nanortalik.
No hotel noted.
Retrieving Results for Index 4: Kenitra.
No hotel noted.
Retrieving Results for Index 5: Los Llanos de Aridane.
No hotel noted.
Retrieving Results for Index 6: Caravelas.
No hotel noted.
Retrieving Results for Index 7: Sitka.
No hotel noted.
Retrieving Results for Index 8: Carnarvon.
No hotel noted.
Retrieving Results for Index 9: Lebu.
No hotel noted.
Retrieving Results for Index 10: Rosetta.
No hotel noted.
Retrieving Results for Index 11: Rikitea.
No hotel noted.
Retrieving Results for Index 12: Setermoen.
No hotel noted.
Retrieving Results for Index 13: Voh.
No hotel noted.
Retrieving Results for Index 14: Albany.
No hotel noted.
Retrieving Results for Index 15: Barrow.
No hotel noted.
Retrieving Results for Index 16: Bluff.
No hotel noted.
Retriev

In [7]:
Hotel_df.head(11)

,City,Latitude,Longitude,Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date and Time
0,Sobolevo,54.4299,31.8996,42.96,70,12,5.73,RU,2022-05-22 19:21:36
1,Ribeira Grande,38.5167,-28.7000,64.80,100,75,2.30,PT,2022-05-22 19:21:37
2,Amahai,-3.3333,128.9167,77.83,87,99,1.90,ID,2022-05-22 19:21:37
3,Nanortalik,60.1432,-45.2371,37.51,98,100,6.85,GL,2022-05-22 19:21:37
4,Kenitra,34.2610,-6.5802,68.76,77,48,7.49,MA,2022-05-22 19:20:17
5,Los Llanos de Aridane,28.6585,-17.9182,67.75,76,75,16.11,ES,2022-05-22 19:21:38
6,Caravelas,-17.7125,-39.2481,74.57,66,2,6.96,BR,2022-05-22 19:21:38
7,Sitka,57.0531,-135.3300,49.89,81,100,8.05,US,2022-05-22 19:19:35
8,Carnarvon,-24.8667,113.6333,70.86,57,33,18.01,AU,2022-05-22 19:21:39
9,Lebu,-37.6167,-73.6500,52.27,87,100,11.05,CL,2022-05-22 19:19:52


In [28]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
# NOTE: be sure to update with your DataFrame name
Locations = Hotel_df[["Latitude", "Longitude"]]


KeyError: "'Hotel Name'"

In [25]:
markers = gmaps.marker_layer(Locations)
# Add the layer to the map
fig.add_layer(markers)
#fig.add_layer(hotel_layer)
fig

NameError: name 'Locations' is not defined

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(Locations)
# Add the layer to the map
fig.add_layer(markers)
#fig.add_layer(hotel_layer)
fig

# Display figure


NameError: name 'Locations' is not defined